The MIRT.ipynb notebook was not giving good performance so an unaltered vesrion was created to compare

In [1]:
!pip --quiet install EduData
!pip --quiet install EduCDM
# Download the Cognitive Diagnosis Benchmark Datasets (CDBD)
from EduData import get_data

get_data("cdbd-a0910", "../../data")

downloader, INFO http://base.ustc.edu.cn/data/cdbd/a0910/item.csv is saved as ..\..\data\a0910\item.csv
downloader, INFO file existed, skipped


'..\\..\\data'

In [2]:
# Load the data from files
import pandas as pd

train_data = pd.read_csv("../../data/a0910/train.csv")
valid_data = pd.read_csv("../../data/a0910/valid.csv")
test_data = pd.read_csv("../../data/a0910/test.csv")

train_data.head(5)

,user_id,item_id,score
0,1615,12977,1
1,782,13124,0
2,1084,16475,0
3,593,8690,0
4,127,14225,1


In [3]:
len(train_data), len(valid_data), len(test_data)

(186049, 25606, 55760)

In [4]:
# Transform data to torch Dataloader (i.e., batchify)
# batch_size is set to 32

import torch
from torch.utils.data import TensorDataset, DataLoader

batch_size = 256
def transform(x, y, z, batch_size, **params):
    dataset = TensorDataset(
        torch.tensor(x, dtype=torch.int64),
        torch.tensor(y, dtype=torch.int64),
        torch.tensor(z, dtype=torch.float)
    )
    return DataLoader(dataset, batch_size=batch_size, **params)

train, valid, test = [
    transform(data["user_id"], data["item_id"], data["score"], batch_size)
    for data in [train_data, valid_data, test_data]
]
train, valid, test

(<torch.utils.data.dataloader.DataLoader at 0x26203734100>,
 <torch.utils.data.dataloader.DataLoader at 0x262037490a0>)

In [5]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [6]:
from EduCDM import MIRT

cdm = MIRT(4164, 17747, 123)

cdm.train(train, valid, epoch=2)
cdm.save("mirt.params")

Epoch 0: 100%|███████████████████████████████████████████████████████████████████████| 727/727 [00:13<00:00, 52.63it/s]


[Epoch 0] LogisticLoss: 3.900918


evaluating: 100%|███████████████████████████████████████████████████████████████████| 101/101 [00:00<00:00, 239.02it/s]


[Epoch 0] auc: 0.551189, accuracy: 0.578341


Epoch 1: 100%|███████████████████████████████████████████████████████████████████████| 727/727 [00:13<00:00, 55.61it/s]


[Epoch 1] LogisticLoss: 2.480133


evaluating: 100%|███████████████████████████████████████████████████████████████████| 101/101 [00:00<00:00, 234.03it/s]
INFO:root:save parameters to mirt.params


[Epoch 1] auc: 0.554092, accuracy: 0.576388


In [7]:
cdm.load("mirt.params")
auc, accuracy = cdm.eval(test)
print("auc: %.6f, accuracy: %.6f" % (auc, accuracy))

INFO:root:load parameters from mirt.params
evaluating: 100%|███████████████████████████████████████████████████████████████████| 218/218 [00:00<00:00, 222.17it/s]

auc: 0.554692, accuracy: 0.573870
